In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv
import glob

In [2]:

# All files ending with .csv with depth of 2 folder
csv_paths = []
csv_paths = glob.glob("../Complete_Data/*/*.csv")


In [3]:
for i in range(len(csv_paths)):
    Y = np.genfromtxt(csv_paths[i], delimiter=",")
   
    # Calculating the number of readings in a file
    input_file = open(csv_paths[i],"r+")
    reader_file = csv.reader(input_file)
    lines = len(next(reader_file))
    #print(lines)
    ppg_Signal=[]
    BP_Signal=[]
    ECG_Signal=[]
    for i in range(0,lines,500):
        # Extracting row cooresponding to PPG signal because the first row of csv file is PPG signal
        if(i+500<lines):
            ppg_Signal = Y[0,i:i+500]
            # Extracting row cooresponding to BP signal
            BP_Signal  = Y[1,i:i+500]
            # Extracting row cooresponding to ECG signal
            ECG_Signal = Y[2,i:i+500]
            i+=500
        
            """if(len(ppg_Signal)!=500):
                print(len(ppg_Signal))
            """
            
            
            # Diastolic time feature Extraction _______________________
            
            # Find local maximum or local maximum of PPG signals
            
            PPG1=np.gradient(ppg_Signal)
            [peak, loc] = signal.find_peaks(PPG1) #getting maxima or minima

            # Finding the local minimum of the PPG signals
            PPG2=np.gradient(PPG1)
            
            [peak_minimum, loc1] = signal.find_peaks(PPG2) #getting minima
            peak_maximum=[]
            
            #Since we got minima, now the rest values in peak are maximas
            for i in peak:
                if i not in peak_minimum:
                    peak_maximum.append(i)
            
            diastolic_time=[]
            #print(len(peak_maximum))
            
            for i in range (0,min(len(peak_maximum),len(peak_minimum))-1):
                diastolic_time.append(abs(peak_maximum[i]-peak_minimum[i+1]))
            #print(len(peak_maximum),' ',len(peak_minimum))
            
            dia_time = []
            for i in range (4):
                sum=0
                
                for j in range (i*len(diastolic_time)//4,(i+1)*len(diastolic_time)//4):
                    sum=sum+ diastolic_time[i]
                sum=sum//(len(diastolic_time)//4)
                dia_time.append(sum)
            
            with open('../Features_Datasets/diastolic_time.csv', 'a') as f: 
                for i in range(4):
                    f.write(str(dia_time[i])+",")
                    f.write("\n")
            
        else:
            i=i+500    

        
    